In [16]:
import json
from datetime import datetime
from typing import Dict, List, Set, Tuple
from collections import defaultdict
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import os

class NumpyJSONEncoder(json.JSONEncoder):
    """Custom JSON encoder for handling NumPy data types."""
    def default(self, obj):
        if isinstance(obj, np.generic):
            return obj.item()
        return super().default(obj)

class HealthcareAnnotationValidator:
    def __init__(self, schema_path: str = None):
        """
        Initialize validator with schema configuration.
        
        Args:
            schema_path: Path to schema JSON file. If None, uses default path in project directory.
        """
        if schema_path is None:
            # Default to project directory path
            schema_path = r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json'
        
        try:
            with open(schema_path, 'r') as f:
                self.schema = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"Schema file not found at: {schema_path}. Please verify the file exists.")
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON in schema file at: {schema_path}")
        except Exception as e:
            raise Exception(f"Error loading schema file: {str(e)}")
        
        # Initialize schema lookups
        self._init_schema_lookups()
        
        # Initialize KPI tracking
        self.kpi_targets = {
            'inter_annotator_agreement': 0.90,
            'medical_terminology_accuracy': 0.98,
            'service_classification_accuracy': 0.95,
            'provider_identification_accuracy': 0.98,
            'required_fields_completion': 1.0,
            'supporting_info_capture': 0.95,
            'patient_demographic_completion': 0.90,
            'geographic_coverage': 1.0,
            'annotations_per_hour': 40,
            'quality_verification_rate': 1.0,
            'error_correction_time': 24,  # hours
            'documentation_completeness': 1.0
        }

    def _init_schema_lookups(self):
        """Initialize lookup dictionaries for schema validation."""
        try:
            self.valid_entity_types = {
                option['value'] for tool in self.schema['tools'] 
                if tool['name'] == 'entities'
                for classification in tool['classifications']
                if classification['name'] == 'entity_type'
                for option in classification['options']
            }
            
            self.valid_relationship_types = {
                option['value'] for tool in self.schema['tools']
                if tool['name'] == 'relationships'
                for classification in tool['classifications']
                if classification['name'] == 'relationship_type'
                for option in classification['options']
            }
            
            self.valid_service_categories = {
                option['value'] for classification in self.schema['classifications']
                if classification['name'] == 'service_category'
                for option in classification['options']
            }
        except KeyError as e:
            raise ValueError(f"Invalid schema structure. Missing required key: {str(e)}")

    def batch_process_annotations(self, annotations: List[Dict], 
                                reference_annotations: List[Dict] = None,
                                medical_terminology_dict: Dict = None) -> Dict:
        """
        Process multiple annotations in batch and calculate KPIs.
        """
        start_time = time.time()
        
        # Parallel processing of basic validation
        with ThreadPoolExecutor() as executor:
            validation_results = list(executor.map(self.validate_annotation, annotations))
        
        # Calculate KPIs
        kpi_results = self._calculate_kpis(
            annotations,
            validation_results,
            reference_annotations,
            medical_terminology_dict,
            start_time
        )
        
        # Generate summary
        summary = self._generate_summary_report(validation_results, kpi_results)
        
        # Convert NumPy types to Python native types
        summary = json.loads(json.dumps(summary, cls=NumpyJSONEncoder))
        kpi_results = json.loads(json.dumps(kpi_results, cls=NumpyJSONEncoder))
        
        return {
            'validation_results': validation_results,
            'kpi_metrics': kpi_results,
            'summary': summary
        }

    # ... [rest of the class implementation remains the same] ...

# Example usage
if __name__ == "__main__":
    try:
        # Example batch of annotations
        batch_annotations = [
            {
                "entities": [
                    {
                        "id": "e1",
                        "type": "provider_name",
                        "bbox": [10, 10, 100, 30],
                        "text": "Dr. Smith"
                    }
                ],
                "classifications": {
                    "service_category": ["primary_care"],
                    "urgency_level": "routine"
                }
            }
        ]
        
        # Initialize validator
        validator = HealthcareAnnotationValidator()
        
        # Process batch
        results = validator.batch_process_annotations(
            annotations=batch_annotations,
            reference_annotations=None,
            medical_terminology_dict=None
        )
        
        # Generate detailed report
        report_df = validator.generate_detailed_report(results)
        
        # Print summary using the custom JSON encoder
        print("\nValidation Summary:")
        print(json.dumps(results['summary'], indent=2, cls=NumpyJSONEncoder))
        
        print("\nDetailed Report Preview:")
        print(report_df.head())
        
        print("\nRecommendations:")
        for idx, rec in enumerate(results['summary']['recommendations'], 1):
            print(f"{idx}. {rec}")
            
    except Exception as e:
        print(f"Error running validator: {str(e)}")
        raise

Error running validator: 'HealthcareAnnotationValidator' object has no attribute 'validate_annotation'


AttributeError: 'HealthcareAnnotationValidator' object has no attribute 'validate_annotation'

In [10]:
import os
import json

def verify_file_access():
    # Print current working directory
    print(f"Current working directory: {os.getcwd()}")
    
    # Try multiple path variations
    paths_to_try = [
        'labelbox_schema.json',
        './labelbox_schema.json',
        r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json'
    ]
    
    print("\nChecking file existence:")
    for path in paths_to_try:
        exists = os.path.exists(path)
        is_file = os.path.isfile(path)
        print(f"\nPath: {path}")
        print(f"Exists: {exists}")
        print(f"Is file: {is_file}")
        if exists and is_file:
            try:
                with open(path, 'r') as f:
                    json.load(f)
                print("✓ Successfully opened and parsed JSON")
                return path  # Return the first working path
            except json.JSONDecodeError:
                print("✗ File exists but contains invalid JSON")
            except Exception as e:
                print(f"✗ Error reading file: {str(e)}")
    
    return None

if __name__ == "__main__":
    working_path = verify_file_access()
    if working_path:
        print(f"\nSuccess! Use this path: {working_path}")
    else:
        print("\nNo working path found. Please verify file location and permissions.")

Current working directory: C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\eda_output

Checking file existence:

Path: labelbox_schema.json
Exists: False
Is file: False

Path: ./labelbox_schema.json
Exists: False
Is file: False

Path: C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json
Exists: True
Is file: True
✓ Successfully opened and parsed JSON

Success! Use this path: C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json


In [22]:
import json
from datetime import datetime
from typing import Dict, List, Set, Tuple
from collections import defaultdict
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import os

class NumpyJSONEncoder(json.JSONEncoder):
    """Custom JSON encoder for handling NumPy data types."""
    def default(self, obj):
        if isinstance(obj, np.generic):
            return obj.item()
        return super().default(obj)

class HealthcareAnnotationValidator:
    def __init__(self, schema_path: str = None):
        """Initialize validator with schema configuration."""
        if schema_path is None:
            schema_path = r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json'
        
        try:
            with open(schema_path, 'r') as f:
                self.schema = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"Schema file not found at: {schema_path}")
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON in schema file at: {schema_path}")
        except Exception as e:
            raise Exception(f"Error loading schema file: {str(e)}")
        
        self._init_schema_lookups()
        
        self.kpi_targets = {
            'inter_annotator_agreement': 0.90,
            'medical_terminology_accuracy': 0.98,
            'service_classification_accuracy': 0.95,
            'provider_identification_accuracy': 0.98,
            'required_fields_completion': 1.0,
            'supporting_info_capture': 0.95,
            'patient_demographic_completion': 0.90,
            'geographic_coverage': 1.0,
            'annotations_per_hour': 40,
            'quality_verification_rate': 1.0,
            'error_correction_time': 24,
            'documentation_completeness': 1.0
        }

    def _init_schema_lookups(self):
        """Initialize lookup dictionaries for schema validation."""
        try:
            self.valid_entity_types = {
                option['value'] for tool in self.schema['tools'] 
                if tool['name'] == 'entities'
                for classification in tool['classifications']
                if classification['name'] == 'entity_type'
                for option in classification['options']
            }
            
            self.valid_relationship_types = {
                option['value'] for tool in self.schema['tools']
                if tool['name'] == 'relationships'
                for classification in tool['classifications']
                if classification['name'] == 'relationship_type'
                for option in classification['options']
            }
            
            self.valid_service_categories = {
                option['value'] for classification in self.schema['classifications']
                if classification['name'] == 'service_category'
                for option in classification['options']
            }
        except KeyError as e:
            raise ValueError(f"Invalid schema structure. Missing required key: {str(e)}")

    def validate_annotation(self, annotation: Dict) -> Dict:
        """
        Validate a single annotation against the schema.
        
        Args:
            annotation: Dictionary containing annotation data
            
        Returns:
            Dictionary containing validation results
        """
        validation_result = {
            'is_valid': True,
            'errors': [],
            'warnings': []
        }
        
        # Validate entities
        if 'entities' in annotation:
            for entity in annotation['entities']:
                # Check required fields
                required_fields = {'id', 'type', 'bbox', 'text'}
                missing_fields = required_fields - set(entity.keys())
                if missing_fields:
                    validation_result['is_valid'] = False
                    validation_result['errors'].append(
                        f"Missing required fields in entity: {missing_fields}")
                
                # Validate entity type
                if 'type' in entity and entity['type'] not in self.valid_entity_types:
                    validation_result['is_valid'] = False
                    validation_result['errors'].append(
                        f"Invalid entity type: {entity['type']}")
                
                # Validate bbox format
                if 'bbox' in entity:
                    if not (isinstance(entity['bbox'], list) and 
                            len(entity['bbox']) == 4 and 
                            all(isinstance(x, (int, float)) for x in entity['bbox'])):
                        validation_result['is_valid'] = False
                        validation_result['errors'].append(
                            f"Invalid bbox format for entity {entity['id']}")
        
        # Validate classifications
        if 'classifications' in annotation:
            if 'service_category' in annotation['classifications']:
                categories = annotation['classifications']['service_category']
                if not isinstance(categories, list):
                    categories = [categories]
                for category in categories:
                    if category not in self.valid_service_categories:
                        validation_result['is_valid'] = False
                        validation_result['errors'].append(
                            f"Invalid service category: {category}")
        
        return validation_result

    def _calculate_kpis(self, annotations: List[Dict], 
                       validation_results: List[Dict],
                       reference_annotations: List[Dict],
                       medical_terminology_dict: Dict,
                       start_time: float) -> Dict:
        """Calculate KPIs based on validation results and reference data."""
        kpi_results = {}
        
        # Calculate required fields completion rate
        valid_count = sum(1 for result in validation_results if result['is_valid'])
        kpi_results['required_fields_completion'] = valid_count / len(annotations)
        
        # Calculate inter-annotator agreement if reference annotations available
        if reference_annotations:
            agreement_scores = []
            for ann, ref in zip(annotations, reference_annotations):
                if 'entities' in ann and 'entities' in ref:
                    ann_labels = [e['type'] for e in ann['entities']]
                    ref_labels = [e['type'] for e in ref['entities']]
                    if ann_labels and ref_labels:
                        agreement = cohen_kappa_score(ann_labels, ref_labels)
                        agreement_scores.append(agreement)
            
            if agreement_scores:
                kpi_results['inter_annotator_agreement'] = np.mean(agreement_scores)
        
        # Calculate medical terminology accuracy if dictionary available
        if medical_terminology_dict:
            correct_terms = 0
            total_terms = 0
            for ann in annotations:
                if 'entities' in ann:
                    for entity in ann['entities']:
                        if entity['type'] in ['diagnosis', 'medication', 'procedure']:
                            total_terms += 1
                            if entity['text'].lower() in medical_terminology_dict:
                                correct_terms += 1
            
            if total_terms > 0:
                kpi_results['medical_terminology_accuracy'] = correct_terms / total_terms
        
        # Calculate processing speed
        processing_time = time.time() - start_time
        kpi_results['annotations_per_hour'] = (len(annotations) / processing_time) * 3600
        
        return kpi_results

    def _generate_summary_report(self, validation_results: List[Dict], 
                               kpi_results: Dict) -> Dict:
        """Generate a summary report of validation results and KPIs."""
        summary = {
            'total_annotations': len(validation_results),
            'valid_annotations': sum(1 for r in validation_results if r['is_valid']),
            'invalid_annotations': sum(1 for r in validation_results if not r['is_valid']),
            'kpi_metrics': kpi_results,
            'recommendations': []
        }
        
        # Generate recommendations based on KPI results
        for metric, value in kpi_results.items():
            if value < self.kpi_targets[metric]:
                summary['recommendations'].append(
                    f"Improve {metric}: Current {value:.2f} vs Target {self.kpi_targets[metric]}")
        
        return summary

    def generate_detailed_report(self, results: Dict) -> pd.DataFrame:
        """Generate a detailed report as a pandas DataFrame."""
        report_data = []
        
        for idx, (validation, kpis) in enumerate(zip(
            results['validation_results'], 
            [results['kpi_metrics']] * len(results['validation_results'])
        )):
            report_data.append({
                'annotation_id': idx + 1,
                'is_valid': validation['is_valid'],
                'error_count': len(validation['errors']),
                'warning_count': len(validation['warnings']),
                'errors': '; '.join(validation['errors']),
                'warnings': '; '.join(validation['warnings']),
                **kpis
            })
        
        return pd.DataFrame(report_data)

    def batch_process_annotations(self, annotations: List[Dict], 
                                reference_annotations: List[Dict] = None,
                                medical_terminology_dict: Dict = None) -> Dict:
        """Process multiple annotations in batch and calculate KPIs."""
        start_time = time.time()
        
        # Parallel processing of basic validation
        with ThreadPoolExecutor() as executor:
            validation_results = list(executor.map(self.validate_annotation, annotations))
        
        # Calculate KPIs
        kpi_results = self._calculate_kpis(
            annotations,
            validation_results,
            reference_annotations,
            medical_terminology_dict,
            start_time
        )
        
        # Generate summary
        summary = self._generate_summary_report(validation_results, kpi_results)
        
        # Convert NumPy types to Python native types
        summary = json.loads(json.dumps(summary, cls=NumpyJSONEncoder))
        kpi_results = json.loads(json.dumps(kpi_results, cls=NumpyJSONEncoder))
        
        return {
            'validation_results': validation_results,
            'kpi_metrics': kpi_results,
            'summary': summary
        }

In [30]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
import pandas as pd

# Sample data
accuracy_data = {
    'name': ['Inter-annotator Agreement', 'Medical Terminology', 'Service Classification', 'Provider Identification'],
    'value': [0.92, 0.97, 0.96, 0.99],
    'target': [0.90, 0.98, 0.95, 0.98],
}

completeness_data = {
    'name': ['Required Fields', 'Supporting Info', 'Patient Demographics', 'Geographic Coverage'],
    'value': [100, 94, 92, 100],
    'target': [100, 95, 90, 100],
}

processing_data = {
    'name': ['Annotations/Hour', 'Quality Verification', 'Error Correction (hrs)', 'Documentation'],
    'value': [45, 100, 20, 98],
    'target': [40, 100, 24, 100],
}

trend_data = {
    'month': ['Jan', 'Feb', 'Mar', 'Apr'],
    'accuracy': [0.91, 0.93, 0.94, 0.96],
    'completeness': [95, 96, 97, 98],
    'processing': [38, 41, 43, 45],
}

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Healthcare Annotation KPI Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Current Metrics', children=[
            html.Div([
                html.H2("Annotation Accuracy"),
                dcc.Graph(
                    figure=go.Figure(
                        data=[
                            go.Indicator(
                                mode="number+gauge+delta",
                                value=accuracy_data['value'][0],
                                title={"text": accuracy_data['name'][0]},
                                gauge={
                                    'axis': {'range': [None, 1]},
                                    'bar': {'color': "#4CAF50"},
                                    'steps': [
                                        {'range': [0, 0.9], 'color': "#FFA726"},
                                        {'range': [0.9, 1], 'color': "#4CAF50"},
                                    ],
                                },
                            )
                        ]
                    )
                ),
                html.H2("Data Completeness"),
                dcc.Graph(
                    figure=go.Figure(data=[
                        go.Bar(name='Current', x=completeness_data['name'], y=completeness_data['value']),
                        go.Bar(name='Target', x=completeness_data['name'], y=completeness_data['target']),
                    ]).update_layout(barmode='group')
                ),
                html.H2("Processing Standards"),
                dcc.Graph(
                    figure=go.Figure(data=[
                        go.Bar(name='Current', x=processing_data['name'], y=processing_data['value']),
                        go.Bar(name='Target', x=processing_data['name'], y=processing_data['target']),
                    ]).update_layout(barmode='group')
                ),
            ]),
        ]),
        dcc.Tab(label='Trends', children=[
            html.Div([
                html.H2("KPI Trends Over Time"),
                dcc.Graph(
                    figure=go.Figure(data=[
                        go.Scatter(x=trend_data['month'], y=trend_data['accuracy'], mode='lines+markers', name='Accuracy'),
                        go.Scatter(x=trend_data['month'], y=trend_data['completeness'], mode='lines+markers', name='Completeness'),
                        go.Scatter(x=trend_data['month'], y=trend_data['processing'], mode='lines+markers', name='Processing Speed'),
                    ]).update_layout(title='KPI Trends Over Time')
                ),
            ]),
        ]),
    ]),
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
python app.py